In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import pickle
import json

In [2]:
df = pd.read_csv('Ta_Quynh_Nga.csv')
df

,Full Name,Email,DR-NTU URL,Website URL,DBLP URL,Citations (All)
0,Wai Kin Adams Kong,adamskong@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00834,NaN,https://dblp.org/pid/16/3792.html,NaN
1,Luu Anh Tuan,anhtuan.luu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp01296,https://tuanluu.github.io/,https://dblp.org/pid/81/8329.html,2949.0
2,Anupam Chattopadhyay,anupam@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp01076,https://scholar.google.co.in/citations?user=TI...,https://dblp.org/pid/99/4535.html,4891.0
3,Anwitaman Datta,anwitaman@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00706,https://personal.ntu.edu.sg/anwitaman/,https://dblp.org/pid/d/AnwitamanDatta.html,7673.0
4,Arvind Easwaran,arvinde@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00687,https://cps-research-group.github.io/,https://dblp.org/pid/73/1708.html,2509.0
...,...,...,...,...,...,...
77,Loke Yuan Ren,yrloke@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00585,https://personal.ntu.edu.sg/yrloke/,https://dblp.org/pid/11/9550.html,13.0
78,Zhang Jie,zhangj@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00759,https://personal.ntu.edu.sg/zhangj/,https://dblp.org/pid/84/6889-2.html,10322.0
79,Zinovi Rabinovich,zinovi@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00214,zinovi.net,https://dblp.org/pid/93/4009.html,945.0
80,Liu Ziwei,ziwei.liu@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00203,https://liuziwei7.github.io/,https://dblp.org/pid/05/6300-2.html,18816.0


# Broad Areas

In [3]:
url = 'https://www.ntu.edu.sg/scse/about-us/our-people'
soup_source = requests.get(url).text
soup = BeautifulSoup(soup_source,'lxml')
areas = {}
for row in soup.find("ol").find_all('li'):
    broad_area = row.find('strong').text
    specific_areas = row.text.split(': ')[-1].replace('\n', '').replace('(', ', ').replace(')', ', ')
    specific_areas = specific_areas.split(', ')
    specific_areas = [a for a in specific_areas if a != '' and a != '.']
    specific_areas[-1] = specific_areas[-1].split('.')[0]
    print(f'{broad_area}\n{specific_areas}\n')
    areas[broad_area] = specific_areas
    
areas['Artificial Intelligence'] = ['Artificial Intelligence', 'Deep learning', 'Machine Learning',
                                    'Reinforcement Learning']
areas['Computer Vision & Language'].append('Natural Language Processing')
areas['Data Management & Analytics'].append('Data mining')

Hardware & Embedded Systems
['low-energy hardware', 'hardware acceleration', 'architecture-aware algorithms', 'reconfigurable computing', 'embedded sensing', 'multi-functional sensors', 'IP-enabled devices']

Cyber Security and Forensics
['data privacy', 'privacy-preserving analytics', 'malware analysis', 'formal systems verification', 'mobile device security', 'cyber attack attribution', 'dark web monitoring', 'botnet detection', 'encrypted data processing', 'biometrics', 'cyber forensics']

Data Management & Analytics
['spatio-temporal databases', 'graph-structured databases', 'big data analytics', 'predictive analytics', 'social media analytics', 'question-answer systems', 'sentiment analysis']

Computational Intelligence
['cognitive and neuro systems', 'decision support systems', 'evolutionary', 'memetic and statistical learning', 'fuzzy systems', 'intelligent multi-agents', 'intelligent system and devices', 'machine learning and nature-inspired systems']

Computer Vision & Languag

In [4]:
def find_research_interest(soup):
    research_interest = []
    if not soup.find('div', id='researchinterestsDiv') is None:
        research_interest_text = soup.find('div', id='researchinterestsDiv').text.strip()
        # search in areas
        for k, v in areas.items():
            for area in v:
                if area.lower() in research_interest_text.lower():
                    research_interest.append(area)
        
        # search <br>
        breaks = soup.find('div', id='researchinterestsDiv').get_text(strip=True, separator='\n').splitlines()
        if len(breaks) > 1:
            breaks = [b.replace('•\t', '').replace('- ', '').replace('+ ', '') for b in breaks if len(b) < 70]
            research_interest.extend(breaks)
            print(research_interest)

    return research_interest

In [5]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    url = row['DR-NTU URL']
    print(i, url, row['Full Name'])
    soup_source = requests.get(url).text
    soup = BeautifulSoup(soup_source,'lxml')
    background = soup.find('div', id='biographyDiv').text.strip()

    research_interest = find_research_interest(soup)

    keywords = []
    for item in soup.find_all('span', class_='rkeyword'):
        if item.text.strip() not in ['Computer Science and Engineering', 'Electrical and Electronic Engineering']:
            keywords.append(item.text.strip())

    author_keywords = []
    if not soup.find('div', id='authortagsDiv') is None:
        for item in soup.find('div', id='authortagsDiv').find_all('a'):
            author_keywords.append(item.text.strip())
#     print(keywords, author_keywords, research_interest)
    print()

  0%|          | 0/82 [00:00<?, ?it/s]

0 https://dr.ntu.edu.sg/cris/rp/rp00834 Wai Kin Adams Kong

1 https://dr.ntu.edu.sg/cris/rp/rp01296 Luu Anh Tuan

2 https://dr.ntu.edu.sg/cris/rp/rp01076 Anupam Chattopadhyay
['Computing Architecture', 'Design Automation', 'Security', 'Emerging Technologies']

3 https://dr.ntu.edu.sg/cris/rp/rp00706 Anwitaman Datta

4 https://dr.ntu.edu.sg/cris/rp/rp00687 Arvind Easwaran
['Resource Management in IoT and Edge', 'Safety Assurance in Learning Enabled Components', 'Model-driven Operational Optimization']

5 https://dr.ntu.edu.sg/cris/rp/rp00839 Vun Chan Hua, Nicholas

6 https://dr.ntu.edu.sg/cris/rp/rp00799 Kwoh Chee Keong
['Data mining', 'Data Mining and Analysis on Infectious Disease', 'Heterogeneous Multi-Core Systems For Bioinformatics', 'Constrained Optimzation for Bioinformatics', 'Protein Interaction Network Analysis Using Graph Mining Approaches']

7 https://dr.ntu.edu.sg/cris/rp/rp00963 Yeo Chai Kiat

8 https://dr.ntu.edu.sg/cris/rp/rp00670 Lau Chiew Tong

9 https://dr.ntu.edu.sg/

['Deep learning', 'Machine Learning']

68 https://dr.ntu.edu.sg/cris/rp/rp01275 Vanessa Evers

69 https://dr.ntu.edu.sg/cris/rp/rp01361 Vidya Sudarshan

70 https://dr.ntu.edu.sg/cris/rp/rp00683 Lin Weisi

71 https://dr.ntu.edu.sg/cris/rp/rp01665 Fan Xiuyi

72 https://dr.ntu.edu.sg/cris/rp/rp00564 Liu Yang
['Deep learning', 'Machine Learning']

73 https://dr.ntu.edu.sg/cris/rp/rp01090 Wen Yonggang
['multimedia', 'Machine Learning', 'Cloud Computing and Green Data Centre', 'Multimedia Computing and Communications', 'Machine Learning Systems and Applications']

74 https://dr.ntu.edu.sg/cris/rp/rp00503 He Ying
['Computer graphics', 'Computer-aided design', 'Geometric modeling and processing']

75 https://dr.ntu.edu.sg/cris/rp/rp00572 Li Yi

76 https://dr.ntu.edu.sg/cris/rp/rp00908 Ke Yiping, Kelly
['Data mining', 'Artificial Intelligence', 'Machine Learning']

77 https://dr.ntu.edu.sg/cris/rp/rp00585 Loke Yuan Ren

78 https://dr.ntu.edu.sg/cris/rp/rp00759 Zhang Jie
['Artificial Intelligenc

In [ ]:
[39, 43, 47]

In [205]:
df[df['Full Name'] == 'Miao Chun Yan']

,Full Name,Email,DR-NTU URL,Website URL,DBLP URL,Citations (All)
9,Miao Chun Yan,ascymiao@ntu.edu.sg,https://dr.ntu.edu.sg/cris/rp/rp00084,https://personal.ntu.edu.sg/ascymiao,https://dblp.org/pid/m/ChunyanMiao.html,13007.0


In [129]:
row = df.iloc[37]
url = row['DR-NTU URL']
print(url, row['Full Name'])
soup_source = requests.get(url).text
soup = BeautifulSoup(soup_source,'lxml')
biography = soup.find('div', id='biographyDiv').text.strip()

if not soup.find('div', id='researchinterestsDiv') is None:
    research_interest_text = soup.find('div', id='researchinterestsDiv').text.strip()
    research_interest = find_research_interest(research_interest_text)
    if len(soup.find('div', id='researchinterestsDiv').get_text(strip=True, separator='\n').splitlines()) > 1:
        research_interest.extend(soup.find('div', id='researchinterestsDiv').get_text(strip=True, separator='\n').splitlines())

keywords = []
for item in soup.find_all('span', class_='rkeyword'):
    if item.text.strip() not in ['Computer Science and Engineering', 'Electrical and Electronic Engineering']:
        keywords.append(item.text.strip())

author_keywords = []
if not soup.find('div', id='authortagsDiv') is None:
    for item in soup.find('div', id='authortagsDiv').find_all('a'):
        author_keywords.append(item.text.strip())
print(keywords, author_keywords, research_interest)
print()

https://dr.ntu.edu.sg/cris/rp/rp00772 Long Cheng
[] [] ['big data analytics', '•\tSpatio-temporal data management, mining and learning', '•\tGraph data mining', '•\tBig data analytics', '•\tUrban informatics and computing']



In [123]:
soup.find('div', id='researchinterestsDiv').get_text()

'\n•\tSpatio-temporal data management, mining and learning•\tGraph data mining•\tBig data analytics•\tUrban informatics and computing\n'

In [102]:
def find_research_interest(research_interest):
    ans = []
    for k, v in areas.items():
        for area in v:
            print(area)
            if area.lower() in research_interest.lower():
                ans.append(area)
    return ans

In [100]:
find_research_interest(research_interest_text)

low-energy hardware
hardware acceleration
architecture-aware algorithms
reconfigurable computing
embedded sensing
multi-functional sensors
IP-enabled devices
data privacy
privacy-preserving analytics
malware analysis
formal systems verification
mobile device security
cyber attack attribution
dark web monitoring
botnet detection
encrypted data processing
biometrics
cyber forensics
spatio-temporal databases
graph-structured databases
big data analytics
predictive analytics
social media analytics
question-answer systems
sentiment analysis
cognitive and neuro systems
decision support systems
evolutionary
memetic and statistical learning
fuzzy systems
intelligent multi-agents
intelligent system and devices
machine learning and nature-inspired systems
computer vision
multimedia
image processing
video analytics
speech and signal processing
3D geometric modeling
fringe pattern analysis
machine translation
Natural Language Processing
human computer interaction
user experience design
augmented r

[]

## Subtopics in broad topics

In [3]:
from collections import OrderedDict

url = 'https://scholar.google.com/citations?view_op=top_venues&hl=en&vq=eng'
soup_source = requests.get(url).text
soup = BeautifulSoup(soup_source,'lxml')
subtopics = OrderedDict()
for div in soup.find_all('div', class_='gs_md_ul'):
    for topic in div.find_all('a'):
        subtopics[topic.text] = {'link': 'https://scholar.google.com' + topic['href']}

del subtopics['Subcategories']
del subtopics['Engineering & Computer Science (general)']

In [4]:
len(subtopics)

55

In [5]:
for k, v in tqdm(subtopics.items()):
    url = v['link']
    soup_source = requests.get(url).text
    soup = BeautifulSoup(soup_source,'lxml')
    print(k)
    v['publications'] = []
    for row in soup.find('table', id='gsc_mvt_table').find_all('tr')[1:]:   
        title = row.find_all('td')[1].text.strip()
        v['publications'].append(title)
        
#         if '(' in title:
#             acronym = title.split('(')[1].split(')')[0]
#             print(f'\t{title} {acronym}')
#         else:
#             # find acronym
#             conference_portal_url = f'http://portal.core.edu.au/conf-ranks/?search={title.replace(" ", "%20")}&by=all&source=CORE2021&sort=atitle&page=1'
#             soup_source_conf = requests.get(conference_portal_url).text
#             soup_conf = BeautifulSoup(soup_source_conf,'lxml')
#             if len(soup_conf.find_all('table')) == 1:
#                 acronym = soup_conf.find_all('tr')[1].find_all('td')[1].text.strip()
#                 print(f'\t{title} {acronym}')
#                 v['publications'].append((title, acronym))

  0%|          | 0/55 [00:00<?, ?it/s]

Architecture
Artificial Intelligence
Automation & Control Theory
Aviation & Aerospace Engineering
Bioinformatics & Computational Biology
Biomedical Technology
Biotechnology
Ceramic Engineering
Civil Engineering
Combustion & Propulsion
Computational Linguistics
Computer Graphics
Computer Hardware Design
Computer Networks & Wireless Communication
Computer Security & Cryptography
Computer Vision & Pattern Recognition
Computing Systems
Data Mining & Analysis
Databases & Information Systems
Educational Technology
Environmental & Geological Engineering
Evolutionary Computation
Food Science & Technology
Fuzzy Systems
Game Theory and Decision Science
Human Computer Interaction
Library & Information Science
Manufacturing & Machinery
Materials Engineering
Mechanical Engineering
Medical Informatics
Metallurgy
Microelectronics & Electronic Packaging
Mining & Mineral Resources
Multimedia
Nanotechnology
Ocean & Marine Engineering
Oil, Petroleum & Natural Gas
Operations Research
Plasma & Fusion
Power

In [6]:
def publication_to_subtopic(publ):
    if publ == 'UNK':
        return 'Unknown'
    for topic, v in subtopics.items():
        if topic.lower() in publ.lower():
            return topic
        for p in v['publications']:
            if p.lower() in publ.lower() or publ.lower() in p.lower():
                return topic
    if 'biology' or 'biometric' in publ.lower():
        return 'Bioinformatics & Computational Biology'
    if 'security' in publ.lower():
        return 'Computer Security & Cryptography'
    return 'Others'

In [7]:
print(publication_to_subtopic('Artificial Intelligence asfj'))

Artificial Intelligence


In [8]:
# for i, row in df.iterrows():
#     url = row['DBLP URL']
#     if type(url) != str:    # not NaN
#         continue
        
#     soup_source = requests.get(url).text
#     soup = BeautifulSoup(soup_source,'lxml')
#     for s in soup.find('div', id='refine-by venue')

In [54]:
df.iloc[53]

Full Name                                Lana Obraztsova
Email                                    lana@ntu.edu.sg
DR-NTU URL         https://dr.ntu.edu.sg/cris/rp/rp00219
Website URL                                          NaN
DBLP URL                                             NaN
Citations (All)                                      NaN
Name: 53, dtype: object

In [32]:
def publication_dict(authors, title, venue, venue_url, year, publ_type, cite_html):
    ans = {}
    ans['authors'] = authors
    ans['title'] = title
    ans['venue'] = venue
    ans['venue_url'] = venue_url
    ans['cite_html'] = cite_html.text.strip()
    ans['year'] = int(year)
    ans['type'] = publ_type
    ans['subtopic'] = publication_to_subtopic(venue)
    return ans
        
def get_citations(self):
    url = f'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q={self["title"]}'
    soup_source = requests.get(url).text
    soup = BeautifulSoup(soup_source, 'lxml')
    cite_tag = soup.find('div', class_='gs_fl')

    if cite_tag is None:
        return 0
    cite = cite_tag.text
    if 'Cited by' in cite:
        tmp = re.search(r'\d+', cite) 
        if tmp is None :
            return 0
        else :
            return int(tmp.group())
    return 0
    
def venue_quality(self):
    if '(' in self['venue'] and self['venue'].split('(')[1].split(')')[0] != '':
        venue = self['venue'].split('(')[1].split(')')[0]
        url = f'http://portal.core.edu.au/conf-ranks/?search={venue.replace(" ", "%20")}&by=all&source=all&sort=atitle&page=1'
    else:
        url = f'http://portal.core.edu.au/conf-ranks/?search={self["venue"].replace(" ", "%20")}&by=all&source=all&sort=atitle&page=1'
    soup_source = requests.get(url).text
    soup = BeautifulSoup(soup_source, 'lxml')
    
    def refine_string(s):
        s = re.sub(r"[()\"#/@;:<>{}`+=~|.!?,]", "", s)
        return s
    
    if len(soup.find_all('table')) == 1:
        for row in soup.find_all('tr')[1:]:
            title = row.find_all('td')[0].text.strip()
            acronym = row.find_all('td')[1].text.strip()
            rank = row.find_all('td')[3].text.strip()
            if venue == title:
                return rank
            elif venue.upper() == acronym:
                return rank
            elif acronym.upper() in [refine_string(v).upper() for v in venue.split()]:
                return rank
            elif venue.upper() in [refine_string(t).upper() for t in title.split()]:
                return rank
    return 'unranked'

In [9]:
# def refine_string(s):
#         s = re.sub(r"[()\"#/@;:<>{}`+=~|.!?,]", "", s)
#         return s
    
# text = 'Annual Meeting of the Association for Computational Linguistics (ACL)'
# [refine_string(v).upper() for v in text.split()]

In [281]:
for i, pub in enumerate(publications):
    print(i, pub.type, pub.venue)

0 book Wireless Networks
1 article Advanced Engineering Informatics
2 article IEEE Internet of Things Journal
3 article IEEE Internet of Things Journal
4 article Knowledge-Based Systems
5 article Pattern Recognition
6 article Proceedings of the VLDB Endowment
7 article Proceedings of the VLDB Endowment
8 article IEEE Transactions on Affective Computing
9 article IEEE Transactions on Cloud Computing
10 article IEEE Transactions on Emerging Topics in Computational Intelligence
11 article IEEE Transactions on Industrial Informatics
12 article IEEE Transactions on Image Processing
13 article ACM Transactions on Intelligent Systems and Technology (TIST)
14 article IEEE Transactions on Parallel and Distributed Systems (TPDS)
15 article IEEE Transactions on Parallel and Distributed Systems (TPDS)
16 article IEEE Transactions on Vehicular Technology
17 inproceedings AAAI Conference on Artificial Intelligence (AAAI)
18 inproceedings AAAI Conference on Artificial Intelligence (AAAI)
19 inproceed

In [24]:
def get_publications(url):
    soup_source = requests.get(url).text
    soup = BeautifulSoup(soup_source,'lxml')

    year = None
    publications = []
    for ul in tqdm(soup.find('div', id='publ-section').find_all('ul', class_='publ-list'), desc='Years'):
        for li in ul.find_all('li'):
            if 'class' in li.attrs and 'year' in li.attrs['class']:
                year = int(li.text.strip())
                #print('\n', year)

            # loop for every publication
            elif 'class' in li.attrs and 'entry' in li.attrs['class']:
                # find type
                publ_type = li.attrs['class'][1]

                cite = li.find('cite', class_='data tts-content')
                # find title
                title = cite.find('span', class_='title').text.strip()
                # find authors
                authors = []
                for author in cite.find_all('span', itemprop='author'):
                    authors.append(author.text.strip())

                # find venue
                if cite.find('span', itemprop='isPartOf') != None:
                    #venue = cite.find('span', itemprop='isPartOf').find('span', itemprop='name').text.strip()
                    for child in cite.children:
                        if child.name == 'a':
                            venue_url = child['href']
                            venue_prefix = '/'.join(venue_url.split('/')[:-1]) + '/index.html'
                            soup_source_venue = requests.get(venue_prefix).text
                            soup_venue = BeautifulSoup(soup_source_venue,'lxml')
                            venue_name = soup_venue.find('header', id='headline').find('h1').text.strip()
                            #print(publ_type, child['href'], venue_name)
                else:
                    venue_url, venue_name = None, 'UNK'
                publications.append(publication_dict(authors, title, venue_name, venue_url, year, publ_type, cite))
    return publications

In [25]:
df_ = df.iloc[:]

for i, row in tqdm(df_.iterrows(), total=len(df_), desc='Faculty Members'):
    url = row['DBLP URL']
    if type(url) != str:
        #publication_set.append([])
        publications = []
        with open(f'publication_set/publications_{i}.pkl', 'wb') as file:
            pickle.dump(publications, file)
    else:
        publications = get_publications(url)
        #publication_set.append(publications)
        with open(f'publication_set/publications_{i}.pkl', 'wb') as file:
            pickle.dump(publications, file)

Faculty Members:   0%|          | 0/82 [00:00<?, ?it/s]

Years:   0%|          | 0/3 [00:00<?, ?it/s]

Years:   0%|          | 0/3 [00:00<?, ?it/s]

Years:   0%|          | 0/3 [00:00<?, ?it/s]

Years:   0%|          | 0/3 [00:00<?, ?it/s]

...

Years:   0%|          | 0/3 [00:00<?, ?it/s]

Years:   0%|          | 0/3 [00:00<?, ?it/s]

Years:   0%|          | 0/3 [00:00<?, ?it/s]

Years:   0%|          | 0/2 [00:00<?, ?it/s]

Years:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
df.loc[53, 'DBLP URL']

nan

In [ ]:
import sys
sys.setrecursionlimit(10000)

with open('publication_set.pkl', 'wb') as file:
    pickle.dump(publication_set, file)

In [28]:
with open('subtopics.pkl', 'wb') as file:
    pickle.dump(subtopics, file)

In [33]:
citations = []
for i in tqdm(range(82)):
    with open(f'publication_set/publications_{i}.pkl', 'rb') as f:
        publications = pickle.load(f)
        citations.append([])
    for pub in publications:
        citations[i].append(get_citations(pub))

  0%|          | 0/82 [00:00<?, ?it/s]

In [34]:
with open('citations.pkl', 'wb') as file:
    pickle.dump(citations, file)

In [ ]:
venue_rankings = []
for i in tqdm(range(82)):
    with open(f'publication_set/publications_{i}.pkl', 'rb') as f:
        publications = pickle.load(f)
        venue_rankings.append([])
    for pub in publications:
        venue_rankings[i].append(venue_quality(pub))

In [186]:
def publication_type_full_name(type_):
    convert_dict = {
        'incollection' : 'Parts in Books or Collections',
        'book': 'Books and Theses',
        'article': 'Journal Articles',
        'inproceedings' : 'Conference and Workshop Papers',
        'informal': 'Informal Publications',
    }
    return convert_dict[type_]

def publication_count_by_type(publications):
    count = {}
    for publ in publications:
        if publ.type not in count:
            count[(publ.type)] = 0
        count[(publ.type)] += 1
    return count

def publication_count_by_year(publications):
    count = {}
    for publ in publications:
        if publ.year not in count:
            count[publ.year] = 0
        count[publ.year] += 1
    return count

def publication_count_by_venue(publications):
    count = {}
    for publ in publications:
        if publ.venue not in count:
            count[publ.venue] = 0
        count[publ.venue] += 1
    return count

publication_count_by_type(publications)

{'article': 31,
 'inproceedings': 59,
 'informal': 8,
 'incollection': 2,
 'book': 1}

In [174]:
def publication_count_by_subtopic(publications):
    count = {}
    for publ in publications:
        if publ.subtopic not in count:
            count[publ.subtopic] = 0
        count[publ.subtopic] += 1
    return count
publication_count_by_subtopic(publications)

{'Bioinformatics & Computational Biology': 43,
 'Computer Vision & Pattern Recognition': 37,
 'Artificial Intelligence': 5,
 'Medical Informatics': 2,
 'Multimedia': 2,
 'Computer Security & Cryptography': 6,
 'Evolutionary Computation': 2,
 'Signal Processing': 2,
 'Unknown': 1,
 'Automation & Control Theory': 1}

In [170]:
topic_count = {}

count = publication_count_by_venue(publications)
for k, v in count.items():
    k = publication_to_subtopic(k)
    if k not in topic_count:
        topic_count[k] = 0
    topic_count[k] += v
for k, v in topic_count.items():
    print(v, '\t|\t', k)

44 	|	 Bioinformatics & Computational Biology
37 	|	 Computer Vision & Pattern Recognition
5 	|	 Artificial Intelligence
2 	|	 Medical Informatics
2 	|	 Multimedia
6 	|	 Computer Security & Cryptography
2 	|	 Evolutionary Computation
2 	|	 Signal Processing
1 	|	 Automation & Control Theory


In [45]:
li.attrs['class']

['entry', 'article', 'toc']